In [112]:
import sys

sys.path.append('/root/lib/')

import ROOT as r
import os
import json
import pandas as pd
import uproot 
import awkward as ak
import array as arr
import numpy as np
import shutil

sys.path.append(os.getcwd() + '/../utilities/')
from milliqanProcessor import *
from milliqanScheduler import *
from milliqanCuts import *
from milliqanPlotter import *

%jsroot on
%load_ext autoreload
%autoreload 2

In [28]:
def loadJson(jsonFile):
    fin = open(jsonFile)
    data = json.load(fin)
    lumis = pd.DataFrame(data['data'], columns=data['columns'])
    return lumis

goodRunsName = '/eos/experiment/milliqan/Configs/goodRunsList.json'
lumisName = '/eos/experiment/milliqan/Configs/mqLumis.json'
shutil.copy(goodRunsName, 'goodRunsList.json')
shutil.copy(lumisName, 'mqLumis.json')

goodRuns = loadJson('goodRunsList.json')
lumis = loadJson('mqLumis.json')

FileNotFoundError: [Errno 2] No such file or directory: '/eos/experiment/milliqan/Configs/goodRunsList.json'

In [29]:
#get list of files to look at
files = []

beam = True

dataDir = '/store/user/milliqan/trees/v34/1100/'

for ifile, filename in enumerate(os.listdir(dataDir)):
    #if len(files) > 10: break
    if ifile > 100: break
    
    if not filename.endswith('root'): continue
    
    runNum = int(filename.split('Run')[1].split('.')[0])
    fileNum = int(filename.split('.')[1].split('_')[0])
        
    goodRun = goodRuns['goodRunTight'].loc[(goodRuns['run'] == runNum) & (goodRuns['file'] == fileNum)]
    beamOn = lumis['beam'].loc[(lumis['run'] == runNum) & (lumis['file'] == fileNum)]

    if len(goodRun) == 0 or len(beamOn) == 0: continue

    print(filename, runNum, fileNum, beamOn.item(), goodRun.item())
    
    if beam:
        if goodRun.item() and beamOn.item(): files.append(dataDir+filename)
    else:
        if goodRun.item() and not beamOn.item(): files.append(dataDir+filename)


files, len(files)

NameError: name 'goodRuns' is not defined

In [16]:
@mqCut
def pulseTime(self):
    events = self.events
    straightPath = self.events['straightLineCutPulse']

    timeToUse = 'timeFit_module_calibrated_corrected'

    self.events['straightPathL0Time'] = events[timeToUse][events['layer0'] & events['straightLineCutPulse']]
    self.events['straightPathL1Time'] = events[timeToUse][events['layer1'] & events['straightLineCutPulse']]
    self.events['straightPathL2Time'] = events[timeToUse][events['layer2'] & events['straightLineCutPulse']]
    self.events['straightPathL3Time'] = events[timeToUse][events['layer3'] & events['straightLineCutPulse']]

    '''height0 = ak.max(self.events['area'][events['layer0'] & events['straightLineCutPulse']], axis=1)
    height1 = ak.max(self.events['area'][events['layer1'] & events['straightLineCutPulse']], axis=1)
    height2 = ak.max(self.events['area'][events['layer2'] & events['straightLineCutPulse']], axis=1)
    height3 = ak.max(self.events['area'][events['layer3'] & events['straightLineCutPulse']], axis=1)

    mask0 = (self.events['area'][events['layer0'] & events['straightLineCutPulse']] == height0)
    mask1 = (self.events['area'][events['layer1'] & events['straightLineCutPulse']] == height1)
    mask2 = (self.events['area'][events['layer2'] & events['straightLineCutPulse']] == height2)
    mask3 = (self.events['area'][events['layer3'] & events['straightLineCutPulse']] == height3)

    self.events['straightPathL0Time'] = self.events['straightPathL0Time'][mask0]
    self.events['straightPathL1Time'] = self.events['straightPathL1Time'][mask1]
    self.events['straightPathL2Time'] = self.events['straightPathL2Time'][mask2]
    self.events['straightPathL3Time'] = self.events['straightPathL3Time'][mask3]'''

    self.events['straightPathDiffL03'] = self.events['straightPathL3Time'] - self.events['straightPathL0Time']
    print("testing", ak.drop_none(self.events['straightPathDiffL03']))

    '''output = self.events['straightPathL0Time']
    drop_empty = ak.num(output) > 0
    print("output", ak.drop_none(output[drop_empty]))'''


In [3]:
'''@mqCut
def centralTime(self):
    events = self.events
    timeCut = (events['timeFit_module_calibrated'] > 1100) & (events['timeFit_module_calibrated'] < 1400)
    drop_empty = ak.num(timeCut) > 0
    #newEvents = events[timeCut]
    newEvents = drop_empty & timeCut
    #print()
    #self.events = events[newEvents]
    for branch in branches:
        #print(branch, type(branch))
        self.events[branch] = self.events[branch][newEvents]
'''

"@mqCut\ndef centralTime(self):\n    events = self.events\n    timeCut = (events['timeFit_module_calibrated'] > 1100) & (events['timeFit_module_calibrated'] < 1400)\n    drop_empty = ak.num(timeCut) > 0\n    #newEvents = events[timeCut]\n    newEvents = drop_empty & timeCut\n    #print()\n    #self.events = events[newEvents]\n    for branch in branches:\n        #print(branch, type(branch))\n        self.events[branch] = self.events[branch][newEvents]\n"

In [4]:
@mqCut
def threeInLine(self):
    npeCut = 100

    for path in range(16):

        row = path//4
        col = path%4

        outputName = 'threeHitPath{}'.format(path)

        #print("Checking three in line for path {}, row {}, col {}, with area cut {}".format(path, row, col, areaCut))

        layer0 = (self.events.layer0) & (self.events.nPE >= npeCut) & (self.events.row == row) & (self.events.column == col)
        layer1 = (self.events.layer1) & (self.events.nPE >= npeCut) & (self.events.row == row) & (self.events.column == col)
        layer2 = (self.events.layer2) & (self.events.nPE >= npeCut) & (self.events.row == row) & (self.events.column == col)
        layer3 = (self.events.layer3) & (self.events.nPE >= npeCut) & (self.events.row == row) & (self.events.column == col)

        threeStraight_f0 = ak.any(layer1, axis=1) & ak.any(layer2, axis=1) & ak.any(layer3, axis=1)
        threeStraight_f1 = ak.any(layer0, axis=1) & ak.any(layer2, axis=1) & ak.any(layer3, axis=1)
        threeStraight_f2 = ak.any(layer0, axis=1) & ak.any(layer1, axis=1) & ak.any(layer3, axis=1)
        threeStraight_f3 = ak.any(layer0, axis=1) & ak.any(layer1, axis=1) & ak.any(layer2, axis=1)

        self.events[outputName+'_f0'] = threeStraight_f0
        self.events[outputName+'_f1'] = threeStraight_f1
        self.events[outputName+'_f2'] = threeStraight_f2
        self.events[outputName+'_f3'] = threeStraight_f3

        self.events[outputName+'_s0'] = (threeStraight_f0) & (self.events.layer0) & (self.events.nPE >= npeCut)
        self.events[outputName+'_s1'] = (threeStraight_f1) & (self.events.layer1) & (self.events.nPE >= npeCut)
        self.events[outputName+'_s2'] = (threeStraight_f2) & (self.events.layer2) & (self.events.nPE >= npeCut)
        self.events[outputName+'_s3'] = (threeStraight_f3) & (self.events.layer3) & (self.events.nPE >= npeCut)
        

@mqCut
def firstPulseCut(self, cutName='firstPulse', cut=False, branches=None):

    self.events[cutName] = self.events.ipulse == 0

    if cut:
        for branch in branches:
            self.events[branch] = self.events[branch][self.events[cutName]]

In [5]:
'''@mqCut
def heightCutMod(self, cutName='heightCut', heightCut=800, cut=True, branches=None):
    self.events[cutName] = self.events['height'] >= int(heightCut)
    print(self.events[cutName])
    print(self.events['height'][self.events[cutName]])
    if cut:
        print("In cut", branches)
        for branch in branches:
            print("branch", branch)
            self.events[branch] = self.events[branch][self.events[cutName]]'''

'@mqCut\ndef heightCutMod(self, cutName=\'heightCut\', heightCut=800, cut=True, branches=None):\n    self.events[cutName] = self.events[\'height\'] >= int(heightCut)\n    print(self.events[cutName])\n    print(self.events[\'height\'][self.events[cutName]])\n    if cut:\n        print("In cut", branches)\n        for branch in branches:\n            print("branch", branch)\n            self.events[branch] = self.events[branch][self.events[cutName]]'

need to figure out how to select the max pulse in the time window only

In [6]:
files = []

dir = '../skim/'
for filename in os.listdir(dir):
    if "skim" in filename and filename.endswith('.root'):
        files.append(dir+filename)

files

['../skim/MilliQan_Run1100_v34_skim.root',
 '../skim/MilliQan_Run1300_v34_skim.root',
 '../skim/MilliQan_Run1500_v34_skim_correction.root',
 '../skim/MilliQan_Run1510_v34_skim.root',
 '../skim/MilliQan_Run1500_v34_skim.root',
 '../skim/MilliQan_Run1000_v34_skim.root',
 '../skim/MilliQan_Run1200_v34_skim.root',
 '../skim/MilliQan_Run1400_v34_skim.root',
 '../skim/MilliQan_Run1100_v35_skim.root']

In [85]:
#define a file list to run over
filelist = ['MilliQan_Run1000_v34_skim_correction.root']

#define the necessary branches to run over
branches = ['event', 'tTrigger', 'boardsMatched', 'pickupFlag', 'fileNumber', 'runNumber', 'type', 'ipulse', 'nPE', 'npulses', 'pickupFlagTight',
            'time_module_calibrated', 'timeFit_module_calibrated', 'timeFit_module_calibrated_corrected', 'row', 'column', 'layer', 'height', 'area']

#define the milliqan cuts object
mycuts = milliqanCuts()

#require pulses are not pickup
pickupCut = mycuts.getCut(mycuts.pickupCut, 'pickupCut', cut=True, tight=True, branches=branches)

#require that all digitizer boards are matched
boardMatchCut = mycuts.getCut(mycuts.boardsMatched, 'boardMatchCut', cut=True, branches=branches)

#height cut to get large pulses
muonHeightCut = mycuts.getCut(mycuts.heightCut, 'muonHeightCut', heightCut=1200, cut=True, branches=branches)

#muon area cut
muonAreaCut = getCutMod(mycuts.areaCut, mycuts, 'muonAreaCut', areaCut=500000, cut=True, branches=branches)

#central time cut
centralTimeCut = getCutMod(mycuts.centralTime, mycuts, 'centralTimeCut', cut=True, branches=branches)

#require pulses from bars
barCut = getCutMod(mycuts.barCut, mycuts, 'barCut', cut=True, branches=branches)

#muonAreaCut = mycuts.areaCut('muonAreaCut, areaCut=500000', cut=True, branches=branches)
#muonAreaCut = mycuts.getCut(mycuts.areaCut, 'muonAreaCut', areaCut=500000, cut=True, branches=branches)

#define milliqan plotter
myplotter = milliqanPlotter()

#create root histogram 
bins = 400
xmin = 1100
xmax = 1500

h_pulseTime03 = r.TH2F("h_pulseTime03", "Pulse Times Between Layer 0 and 3", bins, xmin, xmax, bins, xmin, xmax)
h_L0Times = r.TH1F('h_L0Times', "Pulse Times Layer 0", 400, 1100, 1500)
h_L3Times = r.TH1F('h_L3Times', "Pulse Times Layer 3", 400, 1100, 1500)
h_TimeDiff = r.TH1F('h_TimeDiff', "Difference in Layer 0 and 3 Times", 100, -50, 50)
h_height = r.TH1F('h_height', "Height of Passing Pulses", 650, 0, 1300)
h_area = r.TH1F('h_area', "Area of Passing Pulses", 1000, 0, 100e4)

#setattr(milliqanCuts, "centralTime", centralTime)
setattr(milliqanCuts, "pulseTime", pulseTime)
#setattr(milliqanCuts, 'firstPulseCut', firstPulseCut)

'''setattr(milliqanCuts, "pulseTime", pulseTime)
setattr(milliqanCuts, "centralTime", centralTime)
setattr(milliqanCuts, "muonHeightCut", muonHeightCut)
#setattr(milliqanCuts, "muonAreaCut", muonAreaCut)'''

firstPulse = getCutMod(mycuts.firstPulseCut, mycuts, 'firstPulseCut', cutName='firstPulseCut', cut=True, branches=branches)


#add root histogram to plotter
myplotter.addHistograms(h_pulseTime03, ['straightPathL0Time', 'straightPathL3Time'])
myplotter.addHistograms(h_L0Times, 'straightPathL0Time')
myplotter.addHistograms(h_L3Times, 'straightPathL3Time')
myplotter.addHistograms(h_TimeDiff, 'straightPathDiffL03')
myplotter.addHistograms(h_height, 'height')
myplotter.addHistograms(h_area, 'area')

#defining the cutflow
#cutflow = [boardMatchCut, pickupCut, 
cutflow = [boardMatchCut, 
            pickupCut, 
            centralTimeCut, 
            barCut,
            firstPulse, 
            muonAreaCut, 
            mycuts.layerCut, 
            mycuts.straightLineCut, 
            mycuts.pulseTime, 
            myplotter.dict['h_pulseTime03'], myplotter.dict['h_height'], myplotter.dict['h_area'],
            myplotter.dict['h_L0Times'], myplotter.dict['h_L3Times'], myplotter.dict['h_TimeDiff']]

#create a schedule of the cuts
myschedule = milliQanScheduler(cutflow, mycuts, myplotter)

#print out the schedule
myschedule.printSchedule()

#create the milliqan processor object
myiterator = milliqanProcessor(filelist, branches, myschedule, step_size=10000, qualityLevel='override')

#run the milliqan processor
myiterator.run()

#save plots
myplotter.saveHistograms("runs_1100_june3.root")

mycuts.getCutflowCounts()

----------------------------
MilliQan Scheduler:
	0. boardMatchCut
	1. pickupCut
	2. centralTimeCut
	3. barCut
	4. firstPulseCut
	5. muonAreaCut
	6. layerCut
	7. straightLineCut
	8. pulseTime
	9. h_pulseTime03
	10. h_height
	11. h_area
	12. h_L0Times
	13. h_L3Times
	14. h_TimeDiff
----------------------------

Chosen quality level:  override 
Chosen verbosity level:  minimal 


Quality check is being overridden. All files will be processed.
MilliQan Processor: Processing event 0...
testing [[], [], [], [], [], [], [-5.19], [], [], ..., [], [], [], [], [], [], [], []]
Number of processed events 2945
----------------------------------Cutflow Table------------------------------------
Cut                       N Passing Events     N Passing Pulses     Cut Applied                   
-----------------------------------------------------------------------------------
boardsMatched             2944                 105458               True                          
pickupCut                 29

Warning in <TROOT::Append>: Replacing existing TH1: h_pulseTime03 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_L0Times (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_L3Times (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_TimeDiff (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_height (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_area (Potential memory leak).


In [101]:
print('pulseTime' in locals())

True


In [10]:
c1 = r.TCanvas("c1", "c1", 600, 600)

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [86]:
h_pulseTime03.SetMarkerSize(10)
h_pulseTime03.SetMarkerStyle(7)
h_pulseTime03.Draw("")
line0 = r.TLine(1200, 1200, 1400, 1400)
line40p = r.TLine(1240, 1200, 1415, 1375)
line40m = r.TLine(1160, 1200, 1360, 1400)
line0.Draw("same")
line40p.Draw("same")
line40m.Draw("same")
c1.Draw()

In [78]:
h_height.Draw()
c1.Draw()

In [79]:
h_area.Draw()
c1.Draw()

In [36]:
h_L0Times.Draw()
c1.Draw()

In [37]:
h_L3Times.Draw()
c1.Draw()

In [87]:
h_TimeDiff.Rebin(2)
h_TimeDiff.Draw()
c1.Draw()

Plot the beam on/off files together

In [31]:
fin_beamOn = r.TFile.Open('layerTimes_april29_timeFit_On.root')
fin_beamOff = r.TFile.Open('layerTimes_april26_timeFit_Off.root')

In [32]:
#get all relevant plots
fin_beamOn.ls()

h_PulseTime_on = fin_beamOn.Get('h_pulseTime03')
h_PulseTime_off = fin_beamOff.Get('h_pulseTime03')

h_timeDiff_on = fin_beamOn.Get('h_TimeDiff')
h_timeDiff_off = fin_beamOff.Get('h_TimeDiff')

TFile**		layerTimes_april29_timeFit_On.root	
 TFile*		layerTimes_april29_timeFit_On.root	
  KEY: TH2F	h_pulseTime03;3	Pulse Times Between Layer 0 and 3 [current cycle]
  KEY: TH2F	h_pulseTime03;2	Pulse Times Between Layer 0 and 3 [backup cycle]
  KEY: TH2F	h_pulseTime03;1	Pulse Times Between Layer 0 and 3 [backup cycle]
  KEY: TH1F	h_L0Times;3	Pulse Times Layer 0 [current cycle]
  KEY: TH1F	h_L0Times;2	Pulse Times Layer 0 [backup cycle]
  KEY: TH1F	h_L0Times;1	Pulse Times Layer 0 [backup cycle]
  KEY: TH1F	h_L3Times;3	Pulse Times Layer 3 [current cycle]
  KEY: TH1F	h_L3Times;2	Pulse Times Layer 3 [backup cycle]
  KEY: TH1F	h_L3Times;1	Pulse Times Layer 3 [backup cycle]
  KEY: TH1F	h_TimeDiff;3	Difference in Layer 0 and 3 Times [current cycle]
  KEY: TH1F	h_TimeDiff;2	Difference in Layer 0 and 3 Times [backup cycle]
  KEY: TH1F	h_TimeDiff;1	Difference in Layer 0 and 3 Times [backup cycle]
  KEY: TH1F	h_height;3	Height of Passing Pulses [current cycle]
  KEY: TH1F	h_height;2	Height of Pa

In [33]:
h_PulseTime_on.SetMarkerStyle(7)
h_PulseTime_on.Draw()
h_PulseTime_off.SetMarkerColor(2)
h_PulseTime_off.SetMarkerStyle(7)
h_PulseTime_off.Draw("same")
c1.Draw()

In [35]:
h_timeDiff_off.Rebin(2)
h_timeDiff_on.Rebin(2)
h_timeDiff_off.SetLineColor(2)
h_timeDiff_on.Draw()

h_timeDiff_off.Draw("same")
c1.Draw()

In [30]:
fin = r.TFile.Open('MilliQan_Run1000_v34_skim_correction.root')
mytree = fin.Get('t')

In [88]:
allTimes0 = np.array([])
allTimes1 = np.array([])
allTimes2 = np.array([])
allTimes3 = np.array([])
allTimes = []

areaCut = 500000

totalEvents = 0
totalPulses = 0
passesPickup = 0
passesBar = 0
passesArea = 0
passesIpulse = 0
passesCentralTime = 0
pulseCuts = 0
straightCut = 0

for ievent, event in enumerate(mytree):
    mytree.GetEntry(ievent)
    layersHit = [False, False, False, False]
    times0 = []
    times1 = []
    times2 = []
    times3 = []
    times = [times0, times1, times2, times3]
    #if ievent != 0: break

    if not event.boardsMatched: continue

    totalEvents += 1

    for ip in range(len(event.nPE)):

        totalPulses += 1

        #if event.pickupFlagTight[ip]: continue
        #print(event.pickupFlag[ip])
        passesPickup += 1
        if not event.type[ip] == 0: continue
        passesBar += 1
        if not event.area[ip] >= areaCut: continue
        passesArea += 1
        if not event.ipulse[ip] == 0: continue
        passesIpulse += 1
        if not event.timeFit_module_calibrated_corrected[ip] >= 1100 and event.timeFit_module_calibrated_corrected[ip] < 1500: continue
        passesCentralTime += 1

        pulseCuts += 1

        times[event.layer[ip]].append(event.timeFit_module_calibrated_corrected[ip])
        layersHit[event.layer[ip]] = True

    if layersHit[0] == layersHit[1] == layersHit[2] == layersHit[3] == True:
        allTimes.append(times)
        straightCut += 1

print("Total Events {}, total pulses {}".format(totalEvents, totalPulses))
print("Passing pickup {}\n Passing bar cut {}\n passing area cut {}\n Passing ipulse {}\n Passing central time {}".format(passesPickup, passesBar, passesArea, passesIpulse, passesCentralTime))
print("There are {} pulses passing cuts and {} events with straight line paths".format(pulseCuts, straightCut))

Total Events 2944, total pulses 105458
Passing pickup 105458
 Passing bar cut 84291
 passing area cut 14401
 Passing ipulse 14346
 Passing central time 14337
There are 14337 pulses passing cuts and 1398 events with straight line paths


In [77]:
len(allTimes)

8

In [89]:
h_timeDiff = r.TH1F("h_timeDiff", "Time Diff", 100, -50, 50)
c1 = r.TCanvas("c1", "c1", 800, 800)
diffL0L3 = []
for event in allTimes:
    l0Times = event[0]
    l3Times = event[3]
    for l0 in l0Times:
        for l3 in l3Times:
            diff = l3 - l0
            diffL0L3.append(diff)

a_diffL0L3 = arr.array('d', diffL0L3)
h_timeDiff.FillN(len(a_diffL0L3), a_diffL0L3, np.ones(len(a_diffL0L3)))

Warning in <TROOT::Append>: Replacing existing TH1: h_timeDiff (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [90]:
h_timeDiff.Draw()
c1.Draw()

In [162]:
def getLumiofFileList(filelist):

    inputFiles = [getRunFile(x.split('/')[-1]) for x in filelist]

    #mqLumis = shutil.copy('/eos/experiment/milliqan/Configs/mqLumis.json', 'mqLumis.json')
    lumis = pd.read_json('mqLumis.json', orient = 'split', compression = 'infer')

    lumis['start'] = pd.to_datetime(lumis['start'])
    lumis['stop'] = pd.to_datetime(lumis['stop'])

    myfiles = lumis[lumis.apply(lambda row: [int(row['run']), int(row['file'])] in inputFiles, axis=1)]

    totalLumi = myfiles['lumiEst'].sum()

    runTime = getRunTimes(myfiles)

    print("Running over {} files \n total of {} pb^-1 \n total run time {}s".format(len(filelist), totalLumi, runTime))

runTimes = df['stop'] - df['start']

total_time = runTimes.sum()

In [113]:
beamOnRuns=[1200, 1800]
beamOffRuns=[1200, 1800]

lumis = pd.read_json('mqLumis.json', orient = 'split', compression = 'infer')

lumis['start'] = pd.to_datetime(lumis['start'])
lumis['stop'] = pd.to_datetime(lumis['stop'])

beamOn = lumis[(lumis['run'] >= beamOnRuns[0]) & (lumis['run'] < beamOnRuns[1]) & (lumis['beamInFill']==True)]
beamOff = lumis[(lumis['run'] >= beamOffRuns[0]) & (lumis['run'] < beamOffRuns[1]) & (lumis['beamInFill']==False)]

totalLumi = beamOn['lumiEst'].sum()

runTimeOn = (beamOn['stop'] - beamOn['start']).sum().total_seconds()
runTimeOff = (beamOff['stop'] - beamOff['start']).sum().total_seconds()

print("Beam on runs ran for {}s with lumi {}pb^-1".format(runTimeOn, totalLumi))
print("Beam off runs ran for {}s".format(runTimeOff))

scale = runTimeOff/runTimeOn

print("Scale to normalize beam data {}".format(scale))

Beam on runs ran for 10819155.0s with lumi 107787.3515640304pb^-1
Beam off runs ran for 17594042.0s
Scale to normalize beam data 1.6261937276986973


In [152]:
f_beamOn = r.TFile.Open('timingCorrection_beamOn.root', 'READ')
f_beamOff = r.TFile.Open('timingCorrection_beamOff_panelVeto.root', 'READ')

c1 = r.TCanvas("c1", "c1", 800, 600)

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [153]:
h_timeDiff_beamOn = f_beamOn.Get('h_timeDiffOld')
h_timeDiff_beamOff = f_beamOff.Get('h_timeDiffOld')
h_timeDiff_beamOff.SetLineColor(2)

In [154]:
h_timeDiff_beamOn.Scale(1./runTimeOn)
h_timeDiff_beamOff.Scale(1./runTimeOff)
h_timeDiff_beamOn.Draw("hist")
h_timeDiff_beamOn.SetTitle("Time Difference L3-L0")
h_timeDiff_beamOn.GetXaxis().SetTitle("#Delta t (ns)")
h_timeDiff_beamOn.GetYaxis().SetTitle("Events/s")
h_timeDiff_beamOff.Draw("hist same")

f_on = r.TF1('f_on', 'gaus', -20, 20)
f_onMod = r.TF1('f_onMod', 'gaus', -20, 20)
f_off = r.TF1('f_off', 'gaus', -40, 10)

out_on = h_timeDiff_beamOn.Fit(f_on, "0", "", -20, 20)
out_off = h_timeDiff_beamOff.Fit(f_off, "0", "", -40, 10)

f_off.SetLineColor(2)
f_on.SetLineColor(4)
f_on.Draw("same")
f_off.Draw("same")

h_timeDiff_mod = h_timeDiff_beamOn.Clone()
for ibin in range(h_timeDiff_beamOn.GetNbinsX()):
    center = h_timeDiff_beamOn.GetBinCenter(ibin)
    bkgd = f_off.Eval(center)
    if bkgd < 0: bkgd = 0
    beamOnVal = h_timeDiff_beamOn.GetBinContent(ibin)
    fillVal = beamOnVal - bkgd
    print(f"beam val {beamOnVal}, background {bkgd}, fill {fillVal}")
    h_timeDiff_mod.SetBinContent(ibin, fillVal)

out_onMod = h_timeDiff_mod.Fit(f_onMod, "0", "", -20, 20)
f_onMod.SetLineColor(1)
h_timeDiff_mod.SetLineColor(1)
f_onMod.Draw("same")
h_timeDiff_mod.Draw("same")

l1 = r.TLegend(0.2, 0.7, 0.4, 0.9)
l1.AddEntry(h_timeDiff_beamOn, "Beam On #mu {}, #sigma {}".format(round(f_on.GetParameter(1), 2), round(f_on.GetParameter(2), 2)), "l")
l1.AddEntry(h_timeDiff_beamOff, "Beam Off #mu {}, #sigma {}".format(round(f_off.GetParameter(1), 2), round(f_off.GetParameter(2), 2)), "l")
l1.AddEntry(h_timeDiff_mod, "Beam On Mod #mu {}, #sigma {}".format(round(f_onMod.GetParameter(1), 2), round(f_onMod.GetParameter(2), 2)), "l")

l1.Draw()
c1.Draw()

beam val 0.0, background 1.0504657621390645e-228, fill -1.0504657621390645e-228
beam val 0.0, background 2.7165277741398024e-218, fill -2.7165277741398024e-218
beam val 0.0, background 3.948786126861481e-208, fill -3.948786126861481e-208
beam val 0.0, background 3.2264897491480915e-198, fill -3.2264897491480915e-198
beam val 0.0, background 1.4818839078657154e-188, fill -1.4818839078657154e-188
beam val 0.0, background 3.825738567554089e-179, fill -3.825738567554089e-179
beam val 0.0, background 5.551797425650637e-170, fill -5.551797425650637e-170
beam val 0.0, background 4.528654075891988e-161, fill -4.528654075891988e-161
beam val 0.0, background 2.076452017350162e-152, fill -2.076452017350162e-152
beam val 0.0, background 5.351701310375957e-144, fill -5.351701310375957e-144
beam val 0.0, background 7.753165384432706e-136, fill -7.753165384432706e-136
beam val 0.0, background 6.313692764265685e-128, fill -6.313692764265685e-128
beam val 0.0, background 2.8900479994152508e-120, fill -

In [125]:
#check using front+back panel requirement vs just front
f_beamOnPanels = r.TFile.Open('timingCorrection_beamOn_frontBackVeto.root', 'READ')
f_beamOn = r.TFile.Open('timingCorrection_beamOn.root', 'READ')

h_timeDiff_beamOn = f_beamOn.Get('h_timeDiffOld')
h_timeDiff_beamOnPanels = f_beamOnPanels.Get('h_timeDiffOld')
h_timeDiff_beamOnPanels.SetLineColor(2)

h_timeDiff_beamOn.Scale(1./runTimeOn)
h_timeDiff_beamOnPanels.Scale(1./runTimeOn)
h_timeDiff_beamOn.Draw("hist")
h_timeDiff_beamOn.SetTitle("Time Difference L3-L0")
h_timeDiff_beamOn.GetXaxis().SetTitle("#Delta t (ns)")
h_timeDiff_beamOn.GetYaxis().SetTitle("Events/s")
h_timeDiff_beamOnPanels.Draw("hist same")

f_on = r.TF1('f_on', 'gaus', -20, 20)
f_onPanels = r.TF1('f_onPanels', 'gaus', -20, 20)

out_on = h_timeDiff_beamOn.Fit(f_on, "0", "", -10, 20)
out_onPanels = h_timeDiff_beamOnPanels.Fit(f_onPanels, "0", "", -10, 20)

f_off.SetLineColor(2)
f_on.SetLineColor(4)
f_on.Draw("same")
f_onPanels.Draw("same")

l1 = r.TLegend(0.2, 0.7, 0.4, 0.9)
l1.AddEntry(h_timeDiff_beamOn, "Beam On #mu {}, #sigma {}".format(round(f_on.GetParameter(1), 2), round(f_on.GetParameter(2), 2)), "l")
l1.AddEntry(h_timeDiff_beamOnPanels, "Beam On Panels Required #mu {}, #sigma {}".format(round(f_onPanels.GetParameter(1), 2), round(f_onPanels.GetParameter(2), 2)), "l")
l1.Draw()
c1.Draw()
c1.SetLogy(0)

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =       376.96
NDf                       =           11
Edm                       =  2.35433e-08
NCalls                    =           87
Constant                  =  2.50732e-05   +/-   1.18286e-06 
Mean                      =    -0.807953   +/-   0.167089    
Sigma                     =      4.45118   +/-   0.200442     	 (limited)
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =       318.82
NDf                       =           11
Edm                       =  1.66709e-08
NCalls                    =           83
Constant                  =  2.48773e-05   +/-   2.1031e-06  
Mean                      =    -0.535575   +/-   0.23339     
Sigma                     =      3.64767   +/-   0.303117     	 (limited)


In [117]:
#check time differences in beam on/off between individual channels and slab

h_on = f_beamOn.Get('h_timeDiffFrontPanel3')
h_on.Scale(1./runTimeOn)
h_off = f_beamOff.Get('h_timeDiffFrontPanel3')
h_off.Scale(1./runTimeOff)
h_off.SetLineColor(2)

h_on.Draw("hist")
h_off.Draw("hist same")
f_on = r.TF1('f_on', 'gaus', -40, 0)
f_off = r.TF1('f_off', 'gaus', -30, -10)

'''limits_lo = [1.0e2, -1.0e10, 1.0e1, 1.0e1, 1.0e2, 3.0e2]
limits_hi = [1.0e9, 1.0e1, 1.0e3, 2.0e3, 1.0e5, 5.0e4]
for i in range(6):
    f_off.SetParLimits(i, limits_lo[i], limits_hi[i])'''

f_off.SetParLimits(0, 1e-30, 1)
f_off.SetParLimits(1, -30, -10)
f_off.SetParLimits(2, 1e-2, 5)
#f_off.SetParameters(1e-6, -20, 10)

h_on.Fit(f_on, "0", "", -40, 0)
h_off.Fit(f_off, "0", "", -25, -15)

f_off.SetLineColor(2)
f_on.SetLineColor(4)
f_on.Draw("same")
f_off.Draw("same")

c1.Draw()

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      41.1449
NDf                       =           12
Edm                       =  3.13908e-06
NCalls                    =           82
Constant                  =     2.16e-06   +/-   2.51152e-07 
Mean                      =     -14.0739   +/-   0.299813    
Sigma                     =      3.07485   +/-   0.303002     	 (limited)
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      10.8281
NDf                       =            7
Edm                       =  2.28146e-06
NCalls                    =           61
Constant                  =  5.44647e-07   +/-   9.11164e-08  	 (limited)
Mean                      =     -19.5053   +/-   0.421836     	 (limited)
Sigma                     =      2.67897   +/-   0.403051     	 (limited)


In [365]:
c_l1 = r.TCanvas("c_l1", "Layer 1", 1400, 1400)
c_l2 = r.TCanvas("c_l2", "Layer 2", 1400, 1400)
c_l3 = r.TCanvas("c_l3", "Layer 3", 1400, 1400)
c_l4 = r.TCanvas("c_l4", "Layer 4", 1400, 1400)

c_l1.Divide(4, 4)
c_l2.Divide(4, 4)
c_l3.Divide(4, 4)
c_l4.Divide(4, 4)

timeCanvases = [c_l1, c_l2, c_l3, c_l4]

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_l1
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_l2
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_l3
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_l4


In [366]:
boundsOn = [
    [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0],
    [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0],

    [-20, 20], [-20, 20], [-20, 20], [-20, 20], [-20, 20], [-20, 20], [-20, 20], [-20, 20],
    [-20, 20], [-20, 20], [-20, 20], [-20, 20], [-20, 20], [-20, 20], [-20, 20], [-20, 20],

    [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0],
    [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0],

    [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0],
    [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0]
]


boundsOff = [
    [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0],
    [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0],

    [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0],
    [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0],

    [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0],
    [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0],

    [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0],
    [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0], [-40, 0]
]



for i in range(64):

    #if i > 1: break
    h_on = f_beamOn.Get(f'h_timeDiffFrontPanel{i}')
    h_on.Scale(1./runTimeOn)
    h_off = f_beamOff.Get(f'h_timeDiffFrontPanel{i}')
    h_off.Scale(1./runTimeOff)
    h_off.SetLineColor(2)

    f_on = r.TF1('f_on', 'gaus', boundsOn[i][0], boundsOn[i][1])
    f_off = r.TF1('f_off', 'gaus', boundsOff[i][0], boundsOff[i][1])

    '''f_off.SetParLimits(0, 1e-30, 1)
    f_off.SetParLimits(1, -40, -10)
    f_off.SetParLimits(2, 1e-2, 5)'''

    h_on.Fit(f_on, "0", "", boundsOn[i][0], boundsOn[i][1])
    h_off.Fit(f_off, "0", "", boundsOff[i][0], boundsOff[i][1])

    f_off.SetLineColor(2)
    f_on.SetLineColor(4)

    if i%16 == 0:
        timeCanvases[i//16].cd()
    timeCanvases[i//16].cd(i%16+1)

    h_on.Draw("hist")
    h_off.Draw("hist same")
    f_on.Draw("same")
    f_off.Draw("same")


****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      29.8554
NDf                       =           15
Edm                       =  4.14969e-06
NCalls                    =           96
Constant                  =  4.24087e-07   +/-   8.29037e-08 
Mean                      =     -15.6521   +/-   1.36272     
Sigma                     =      6.87487   +/-   1.45623      	 (limited)
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      28.7602
NDf                       =           27
Edm                       =  1.10376e-07
NCalls                    =          106
Constant                  =  1.80111e-07   +/-   2.90929e-08 
Mean                      =     -20.6379   +/-   2.22158     
Sigma                     =      11.7469   +/-   2.73658      	 (limited)
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      32.1573
NDf                   

Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.


In [367]:
c_l1.Draw()

In [368]:
c_l2.Draw()

In [359]:
c_l3.Draw()

In [360]:
c_l4.Draw()